In [ ]:
import sys
import os

import h5py

import numpy as np

from astropy.io import fits

import matplotlib.pyplot as plt

sys.path.append(os.path.abspath('..'))

from mrileyowens.stats import weighted_quantile

In [ ]:
def compare():

    '''
    Compare absolute UV magnitude measurements of the photometric catalog of F775W dropouts from Endsley et al. (2024) by Endsley et al. (2024) and my own
    measurements based on my BEAGLE fits to the catalog
    '''

    # Set common directories
    home = os.getcwd()
    data = f'{home}/data'
    figs = f'{home}/figs'
    results = f'{home}/results'

    # Get the photometric catalog of F775W dropouts from Endsley et al. (2024)
    catalog = fits.open(f'{data}/JADES_f775w_dropouts_Endsley2024.fits')

    # Open my own measurements of the absolute UV magnitude from the BEAGLE fits I made to the Endsley et al. (2024) photometric catalog of F775W dropouts
    with h5py.File(f'{results}/m_uv/endsley2024_f775w_dropouts_beagle_csfh_fits_m_uv.h5', 'r') as f:

        # Get the IDs of the objects
        ids = f.keys()

        # Make a figure to plot the two sets of measurements against each other
        fig, ax = plt.subplots()

        # For each object ID
        for i, id in enumerate(ids):

            # Calculate the object ID's matching index in the Endsley et al. (2024) catalog
            idx_endsley2024 = np.where(catalog[1].data['ID'] == id)

            # Get the median, 16th, and 84th percentiles of the absolute UV magnitudes of Endsley et al. (2024)
            m_uv_endsley2024, m_uv_endsley2024_lower, m_uv_endsley2024_upper = catalog[1].data['Muv'][idx_endsley2024][0], catalog[1].data['Muv_l16'][idx_endsley2024][0], catalog[1].data['Muv_u84'][idx_endsley2024][0]

            # Get the posteriors and corresponding probabilities of my own absolute UV magnitude measurements of the Endsley et al. (2024) F775W dropout catalog
            m_uv_me_posterior, probabilities = f[id]['m_uv'][:], f[id]['probabilities'][:]

            # Calculate the median, 16th, and 84th percentiles of the posteriors of my own absolute UV magnitude measurements
            m_uv_me, m_uv_me_lower, m_uv_me_upper = weighted_quantile(m_uv_me_posterior, probabilities, 0.5), weighted_quantile(m_uv_me_posterior, probabilities, 0.16), weighted_quantile(m_uv_me_posterior, probabilities, 0.84)

            # Plot the two sets of measurements against each other
            ax.errorbar(m_uv_me, m_uv_endsley2024, xerr=[[abs(m_uv_me_lower - m_uv_me)],[abs(m_uv_me_upper - m_uv_me)]], yerr=[[abs(m_uv_endsley2024_lower - m_uv_endsley2024)], [abs(m_uv_endsley2024_upper - m_uv_endsley2024)]], 
                color='black')
            
        # Create a one-to-one line on the figure
        ax.axline((-20,-20), slope=1, color='red', linestyle='dashed')

        # Label the axes
        ax.set_xlabel(r'$M_\text{UV}$ (me)')
        ax.set_ylabel(r'$M_\text{UV}$ (Endsley et al. (2024))')

        # Set the title of the figure
        ax.set_title(r'Comparing $M_\text{UV}$ measurements of the F775W dropouts from Endsley et al. (2024)')

        # Save the figure
        fig.savefig(f'{figs}/m_uv_measurements_comparison.png', dpi=200, bbox_inches='tight')

In [ ]:
compare()